## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,19.53,62,100,19.57,overcast clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,73.18,69,86,1.99,overcast clouds
2,2,Katsuura,JP,35.1333,140.3000,53.76,68,100,12.50,overcast clouds
3,3,Bethel,US,41.3712,-73.4140,29.01,38,100,13.80,overcast clouds
4,4,Vaini,TO,-21.2000,-175.2000,80.76,83,75,11.50,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Vaini,TO,-21.2000,-175.2000,80.76,83,75,11.50,broken clouds
5,5,Vao,NC,-22.6667,167.4833,77.56,71,100,12.66,overcast clouds
15,15,Aku,NG,6.7081,7.3167,81.48,22,20,5.77,few clouds
21,21,Midlothian,US,32.4824,-96.9945,86.34,39,0,18.99,clear sky
27,27,Sisophon,KH,13.5859,102.9737,77.67,81,76,6.08,broken clouds
28,28,Rikitea,PF,-23.1203,-134.9692,78.21,69,77,4.81,broken clouds
31,31,Lamu,KE,-2.2717,40.9020,82.62,85,4,7.70,clear sky
32,32,Avarua,CK,-21.2078,-159.7750,84.25,79,40,11.50,scattered clouds
39,39,Les Cayes,HT,18.2000,-73.7500,82.31,56,90,10.13,overcast clouds
43,43,East London,ZA,-33.0153,27.9116,80.37,45,80,7.47,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Country                213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
5,Vao,NC,77.56,overcast clouds,-22.6667,167.4833,
15,Aku,NG,81.48,few clouds,6.7081,7.3167,
21,Midlothian,US,86.34,clear sky,32.4824,-96.9945,
27,Sisophon,KH,77.67,broken clouds,13.5859,102.9737,
28,Rikitea,PF,78.21,broken clouds,-23.1203,-134.9692,
31,Lamu,KE,82.62,clear sky,-2.2717,40.9020,
32,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,
39,Les Cayes,HT,82.31,overcast clouds,18.2000,-73.7500,
43,East London,ZA,80.37,broken clouds,-33.0153,27.9116,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
702,Port Keats,AU,79.74,broken clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)
704,Santa Rosa,PH,82.11,broken clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
705,Maturin,VE,82.81,overcast clouds,9.7500,-63.1767,Hotel Residencial La Pichincha
707,Aketi,CD,80.46,broken clouds,2.7388,23.7833,
708,Barhi,IN,76.64,clear sky,23.9000,80.8000,"Govt , st senior boys hostel Barhi"


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))